# 데이터 불러오기

In [ ]:
import pandas_datareader.data as web

In [ ]:
df = web.DataReader('005930', 'naver', start='2023-01-01', end='2023-12-31') # 종목번호: 005930
df.head()

,Open,High,Low,Close,Volume
Date,,,,,
2023-01-02,55500,56100,55200,55500,10031448
2023-01-03,55400,56000,54500,55400,13547030
2023-01-04,55700,58000,55600,57800,20188071
2023-01-05,58200,58800,57600,58200,15682826
2023-01-06,58300,59400,57900,59000,17334989


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 245 entries, 2023-01-02 to 2023-12-28
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Open    245 non-null    object
 1   High    245 non-null    object
 2   Low     245 non-null    object
 3   Close   245 non-null    object
 4   Volume  245 non-null    object
dtypes: object(5)
memory usage: 11.5+ KB


In [ ]:
df = df.astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 245 entries, 2023-01-02 to 2023-12-28
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Open    245 non-null    int64
 1   High    245 non-null    int64
 2   Low     245 non-null    int64
 3   Close   245 non-null    int64
 4   Volume  245 non-null    int64
dtypes: int64(5)
memory usage: 11.5 KB


In [ ]:
data = df.to_numpy()
data

array([[   55500,    56100,    55200,    55500, 10031448],
       [   55400,    56000,    54500,    55400, 13547030],
       [   55700,    58000,    55600,    57800, 20188071],
       ...,
       [   76100,    76700,    75700,    76600, 13164909],
       [   76700,    78000,    76500,    78000, 20651042],
       [   77700,    78500,    77500,    78500, 17797536]])

In [ ]:
mins= data.min(axis = 0)
mins

array([  55400,   56000,   54500,   55400, 5824628])

In [ ]:
sizes = data.max(axis = 0) - mins
sizes

array([   22300,    22500,    23000,    23100, 24191593])

In [ ]:
data.shape[0]

245

# 재현성 구현

In [ ]:
import torch
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

# 문제 정의
- 회귀 > 주식데이터 > 입력 길이(seq) = 10일치, 정답데이터 길이: 5일치의 종가

In [ ]:
data.shape[0]

245

- 기간 내 모든 데이터 불러오기

In [ ]:
def transform_data(data, mins, sizes, seq_len = 10, pred_len = 5):
  data = (data -mins) / sizes # 스케일링
  x_list = []
  y_list = []

  for i in range(seq_len, data.shape[0] + 1-pred_len): # 마지막까지 실행해야하므로 1-pred_len > 10~241
    x = data[i-seq_len:i] # 처음부터 10개씩 입력데이터 뽑기
    y = data[i:i+pred_len, 3] # seq_len 이후 5일치, 종가만 뽑기
    x_list.append(x)
    y_list.append(y)

  x_arr = np.array(x_list) # numpy 변환
  y_arr = np.array(y_list) # numpy 변환
  return x_arr, y_arr

In [ ]:
x_arr, y_arr = transform_data(data, mins, sizes)
x_arr.shape, y_arr.shape # numpy로 변환했으므로 3차원형태

((231, 10, 5), (231, 5))

In [ ]:
y_arr # 2차원형태

array([[0.24675325, 0.24242424, 0.21645022, 0.26406926, 0.27705628],
       [0.24242424, 0.21645022, 0.26406926, 0.27705628, 0.34632035],
       [0.21645022, 0.26406926, 0.27705628, 0.34632035, 0.36796537],
       ...,
       [0.77922078, 0.83982684, 0.84848485, 0.88744589, 0.91774892],
       [0.83982684, 0.84848485, 0.88744589, 0.91774892, 0.97835498],
       [0.84848485, 0.88744589, 0.91774892, 0.97835498, 1.        ]])

In [ ]:
y_arr[-1]

array([0.84848485, 0.88744589, 0.91774892, 0.97835498, 1.        ])

In [ ]:
x_arr

array([[[0.0044843 , 0.00444444, 0.03043478, 0.004329  , 0.17389595],
        [0.        , 0.        , 0.        , 0.        , 0.31921842],
        [0.01345291, 0.08888889, 0.04782609, 0.1038961 , 0.59373696],
        ...,
        [0.25112108, 0.23111111, 0.25217391, 0.22077922, 0.26811475],
        [0.25560538, 0.23111111, 0.23478261, 0.22077922, 0.42485557],
        [0.22869955, 0.23111111, 0.25652174, 0.23376623, 0.2763646 ]],

       [[0.        , 0.        , 0.        , 0.        , 0.31921842],
        [0.01345291, 0.08888889, 0.04782609, 0.1038961 , 0.59373696],
        [0.12556054, 0.12444444, 0.13478261, 0.12121212, 0.40750512],
        ...,
        [0.25560538, 0.23111111, 0.23478261, 0.22077922, 0.42485557],
        [0.22869955, 0.23111111, 0.25652174, 0.23376623, 0.2763646 ],
        [0.26457399, 0.24888889, 0.27391304, 0.24675325, 0.1742483 ]],

       [[0.01345291, 0.08888889, 0.04782609, 0.1038961 , 0.59373696],
        [0.12556054, 0.12444444, 0.13478261, 0.12121212, 0.4

In [ ]:
x_arr.shape # batch, seq, features

(231, 10, 5)

# 데이터 클래스 구현

In [ ]:
class FinanceDataset(torch.utils.data.Dataset):
  def __init__(self, x, y=None):
    self.x = x
    self.y = y

  def __len__(self): # 샘플 수 반환
    return len(self.x)

  def __getitem__(self, idx): # 인덱싱+슬라이싱 가능하게 반환
    item = {}
    item['x'] = torch.Tensor(self.x[idx])

    if self.y is not None:
      item['y'] = torch.Tensor(self.y[idx])
    return item

- 결과 확인하기

In [ ]:
dt = FinanceDataset(x_arr, y_arr)
dt[10]

{'x': tensor([[0.2646, 0.2489, 0.2739, 0.2468, 0.1742],
         [0.2601, 0.2444, 0.2652, 0.2424, 0.1656],
         [0.2377, 0.2222, 0.2348, 0.2165, 0.2381],
         [0.2287, 0.2444, 0.2565, 0.2641, 0.2887],
         [0.3004, 0.2800, 0.2870, 0.2771, 0.1580],
         [0.3632, 0.3422, 0.3696, 0.3463, 0.4546],
         [0.3767, 0.3511, 0.3826, 0.3680, 0.3081],
         [0.4036, 0.4000, 0.4087, 0.3983, 0.5347],
         [0.4260, 0.3956, 0.3739, 0.3420, 0.6271],
         [0.3632, 0.3422, 0.2826, 0.2424, 0.9889]]),
 'y': tensor([0.2771, 0.3506, 0.3636, 0.2684, 0.2814])}

In [ ]:
dl = torch.utils.data.DataLoader(dt, batch_size = 1, shuffle = False)
batch = next(iter(dl))
batch # batch, seq, feature 차원 포함

{'x': tensor([[[0.0045, 0.0044, 0.0304, 0.0043, 0.1739],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.3192],
          [0.0135, 0.0889, 0.0478, 0.1039, 0.5937],
          [0.1256, 0.1244, 0.1348, 0.1212, 0.4075],
          [0.1300, 0.1511, 0.1478, 0.1558, 0.4758],
          [0.1928, 0.2089, 0.2217, 0.2294, 0.5297],
          [0.2152, 0.2267, 0.2348, 0.2165, 0.3735],
          [0.2511, 0.2311, 0.2522, 0.2208, 0.2681],
          [0.2556, 0.2311, 0.2348, 0.2208, 0.4249],
          [0.2287, 0.2311, 0.2565, 0.2338, 0.2764]]]),
 'y': tensor([[0.2468, 0.2424, 0.2165, 0.2641, 0.2771]])}

In [ ]:
batch['x'].shape  # batch, seq, feature > batch_size = 1

torch.Size([1, 10, 5])

# RNN Layer
- input_size: 입력 데이터의 피처의 수
- hidden_size: hidden state의 출력 피처의 수
- num_layers: RNN layer의 수(기본값 1)
- batch_first: 입력데이터의 batch 차원 첫번째 여부(기본값 False)
  - True: (batch, seq, feature)
  - False: (seq, batch, feature)
- bidirectional: 양방향 여부(기본값 False)

In [ ]:
SEED =42

In [ ]:
reset_seeds(SEED)
rnn = torch.nn.RNN(x_arr.shape[2], 16) # RNN 객체 생성, 입력데이터 피처 개수: 5, 출력데이터 피처 개수: 16
output, hn= rnn(batch['x'].permute(1,0,2)) # batch_first(default= False) > 변경 후 batch, seq, feature
output.shape # 마지막 모든layer의 각 cell출력값 > seq, batch, feature

torch.Size([10, 1, 16])

In [ ]:
hn.shape # hn > 마지막 cell의 hidden 출력, nlayer, batch, feature

torch.Size([1, 1, 16])

In [ ]:
reset_seeds(SEED)
rnn = torch.nn.RNN(x_arr.shape[2], 16, batch_first = True) # RNN 객체 생성, 입력데이터 피처 개수: 5, 출력데이터 피처 개수: 16
output, hn= rnn(batch['x']) # 각 노드의 output값, 노드(cell)
output.shape,hn.shape # batch, seq, feature

(torch.Size([1, 10, 16]), torch.Size([1, 1, 16]))

In [ ]:
output[:,-1]

tensor([[ 1.4958e-01,  2.2827e-01, -3.2360e-01,  7.3111e-03,  9.2079e-02,
         -1.2976e-01, -2.7482e-01,  8.8964e-03, -2.6018e-01, -2.9190e-01,
          2.2102e-02, -1.3806e-01, -4.2466e-02,  8.5335e-02,  8.2836e-02,
         -1.4650e-02],
        [ 3.6164e-02,  2.9772e-01, -2.1006e-01,  7.4300e-02,  1.6152e-01,
         -7.5715e-02, -4.2863e-01,  1.1225e-01, -2.3765e-01, -4.1362e-01,
          9.2093e-02, -3.1766e-01, -6.0078e-02,  1.2910e-01,  4.9318e-02,
          4.3315e-02],
        [ 4.5722e-02,  2.1667e-01, -1.8708e-01,  1.0839e-01,  1.9736e-01,
         -1.3738e-01, -4.2477e-01,  8.5721e-02, -8.5996e-02, -3.8197e-01,
          8.8760e-02, -4.2051e-01, -4.3496e-02, -8.8838e-03, -1.6403e-02,
          1.0830e-01],
        [ 8.2363e-02,  2.7476e-01, -1.1234e-01,  7.0403e-02,  2.2557e-01,
         -1.0839e-01, -3.7791e-01,  8.6220e-02, -1.1054e-01, -3.2504e-01,
          6.0759e-02, -3.1663e-01, -3.3235e-02,  5.8455e-03, -1.2812e-02,
          4.4357e-02],
        [ 1.1146e-01

In [ ]:
hn

tensor([[[ 0.1060,  0.3206, -0.1388,  0.0777,  0.1697, -0.0974, -0.3326,
           0.1958, -0.0740, -0.2348,  0.0139, -0.2080, -0.0878,  0.0195,
           0.0804, -0.0018]]], grad_fn=<StackBackward0>)

# RNN layer 수
- batch_first: 입력데이터의 batch 차원 첫번째 여부(기본값 False)
  - True: (batch, seq, feature)
  - False: (seq, batch, feature)

In [ ]:
reset_seeds(SEED)

rnn = torch.nn.RNN(x_arr.shape[2], 16, batch_first = True, num_layers= 2) # num_layers(default= 1)
output, hn = rnn(batch['x']) # 각 노드의 output값, 노드(cell)

output.shape, hn.shape # output > batch, seq, feature  hn > nlayer, batch, feature

(torch.Size([1, 10, 16]), torch.Size([2, 1, 16]))

In [ ]:
output[:,-1] # batch, feature

tensor([[-0.0735,  0.0531, -0.0493,  0.2707,  0.4491,  0.0411,  0.3417, -0.1877,
         -0.2657, -0.0089, -0.4345,  0.0164,  0.1962,  0.0152, -0.5012,  0.1802]],
       grad_fn=<SelectBackward0>)

In [ ]:
hn[-1] # batch, feature, num_layers= 2이므로

tensor([[-0.0735,  0.0531, -0.0493,  0.2707,  0.4491,  0.0411,  0.3417, -0.1877,
         -0.2657, -0.0089, -0.4345,  0.0164,  0.1962,  0.0152, -0.5012,  0.1802]],
       grad_fn=<SelectBackward0>)

In [ ]:
hn[-1].shape

torch.Size([1, 16])

# bidirectional
- 양방향 여부(기본값 False)
  - 양방향이면 hidden layer 2개 출력 > feature방향 concatenate

- 각 output은 정방향H와 역방향H의 대응되는 각 cell끼리 concatenate

In [ ]:
reset_seeds(SEED)
rnn = torch.nn.RNN(x_arr.shape[2], 16, bidirectional= True, batch_first = True)
output, hn = rnn(batch['x'])
output.shape, hn.shape # output > batch, seq, featur hn > nlayer, batch, feature

(torch.Size([1, 10, 32]), torch.Size([2, 1, 16]))

In [ ]:
output[:,:1]

tensor([[[ 0.1496,  0.2283, -0.3236,  0.0073,  0.0921, -0.1298, -0.2748,
           0.0089, -0.2602, -0.2919,  0.0221, -0.1381, -0.0425,  0.0853,
           0.0828, -0.0146,  0.0249, -0.2321,  0.1182,  0.0373, -0.4371,
           0.1355, -0.0380, -0.0402, -0.2926, -0.4796,  0.4312, -0.0706,
           0.2065,  0.0690, -0.0905,  0.0258]]], grad_fn=<SliceBackward0>)

In [ ]:
hn.shape

torch.Size([2, 1, 16])

In [ ]:
# nlayer, batch, feature > batch, nlayer, feature > batch, nlayer * feature
hn.permute(1,0,2).flatten(1).shape # nlayer부터 flat해야하므로 flatten(1)

torch.Size([1, 32])

# 신경망 모델클래스 구현(단방향)

In [ ]:
class Net(torch.nn.Module):
  def __init__(self, n_features, hidden_size, pred_len):
    super().__init__()
    self.rnn_layer = torch.nn.RNN(n_features, hidden_size, batch_first = True)
    self.fc_layer = torch.nn.Linear(hidden_size, hidden_size // 2)
    self.relu = torch.nn.ReLU()
    self.output_layer = torch.nn.Linear(hidden_size // 2, pred_len) # 예측 일수: pred_len = 5

  def forward(self, x):
    # output > batch, seq, feature
    # hn > nlayer, batch, feature
    output, hn = self.rnn_layer(x)
    x = hn[-1] # nlayer, batch, feature > batch,feature
    x = self.fc_layer(x)
    x = self.relu(x)
    return self.output_layer(x)

- 결과 확인하기

In [ ]:
model = Net(x_arr.shape[2], 32, 5)
model(batch['x'])

tensor([[ 0.0195, -0.1262, -0.1497,  0.1426,  0.1228]],
       grad_fn=<AddmmBackward0>)

# 학습데이터 train loop함수 구현

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer, device):
  model.train() # 학습 모드 전환
  epoch_loss = 0
  for batch in dataloader: # 배치단위 데이터 반환
    pred = model(batch['x'].to(device)) # 예측
    loss = loss_fn(pred, batch['y'].to(device)) # 손실함수로 계산

    optimizer.zero_grad() # 이전 경사 누적되는 걸 방지하기 위해 기울기 0으로 초기화
    loss.backward() # 역전파
    optimizer.step() # 가중치 업데이트
    epoch_loss += loss.item() # epoch loss를 계산하기 위해 배치 loss 모두 합치기

  epoch_loss /= len(dataloader) # 평균내서 epoch loss 구하기
  return epoch_loss

# 테스트데이터 test loop함수 구현

In [ ]:
@torch.no_grad() # with 사용과 같은 의미
def test_loop(dataloader, model, loss_fn, device):
  epoch_loss = 0
  model.eval() # 평가모드
  pred_list = []
  for batch in dataloader:
    pred = model(batch['x'].to(device))
    if batch.get('y') is not None: # 검증데이터일 경우, y키에 텐서가 있을 경우만 loss계산
      loss = loss_fn(pred, batch['y'].to(device))
      epoch_loss += loss.item()


    pred = pred.to('cpu').numpy()
    pred_list.append(pred)

  epoch_loss /= len(dataloader)
  pred = np.concatenate(pred_list)

  return epoch_loss, pred

# 조합 후 KFold학습 수행

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold

n_splits = 5 # K-Fold의 k값
n_features = x_arr.shape[2]
hidden_size = 16 # rnn layer의 hidden_size(출력데이터 피처 개수)
pred_len = y_arr.shape[1] # 예측 길이(output layer노드 수)
batch_size = 32
epochs = 1000
cv = KFold(n_splits, random_state= SEED, shuffle = True)
loss_fn = torch.nn.MSELoss()
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
is_holdout = False
reset_seeds(SEED)
score_list= []

for i, (tri,vai) in enumerate(cv.split(x_arr)): # cv 학습 진행
  # 학습데이터
  train_dt = FinanceDataset(x_arr[tri], y_arr[tri])
  train_dl = torch.utils.data.DataLoader(train_dt, batch_size = batch_size, shuffle = True)

  # 검증데이터
  valid_dt = FinanceDataset(x_arr[vai], y_arr[vai])
  valid_dl = torch.utils.data.DataLoader(valid_dt, batch_size = batch_size, shuffle = False)

  model = Net(n_features, hidden_size, pred_len).to(device)
  optimizer = torch.optim.Adam(model.parameters())


  best_score = np.inf
  patience = 0
  for _ in tqdm(range(epochs)):
    train_loss = train_loop(train_dl, model, loss_fn, optimizer, device)
    valid_loss, pred = test_loop(valid_dl, model, loss_fn, device)

    pred = pred * sizes[3] + mins[3] # 민맥스 결과값을 원상태로 돌려놓기 > 종가이므로 인덱싱 '3'
    true = y_arr[vai] * sizes[3] + mins[3] # 검증용 정답데이터
    score = mean_absolute_error(true, pred)

    patience += 1
    if score < best_score: # error > 작을수록 좋으므로
      patience = 0
      best_score = score
      torch.save(model.state_dict(), f'model_{i}.pt') # 가중치 저장하기

    if patience == 5:
      break
  print(f'Fold-{i} MAE: {best_score}')
  score_list.append(best_score)
  if is_holdout:
    break

  0%|          | 0/1000 [00:00<?, ?it/s]

Fold-0 MAE: 1099.3488198138298


  0%|          | 0/1000 [00:00<?, ?it/s]

Fold-1 MAE: 1356.7452615489133


  0%|          | 0/1000 [00:00<?, ?it/s]

Fold-2 MAE: 1147.5340862771739


  0%|          | 0/1000 [00:00<?, ?it/s]

Fold-3 MAE: 1057.2127717391304


  0%|          | 0/1000 [00:00<?, ?it/s]

Fold-4 MAE: 1147.7310461956522


In [ ]:
np.mean(score_list)

1161.7143971149399

# 신경망모델 클래스구현(양방향)

In [ ]:
class Net(torch.nn.Module):
  def __init__(self, n_features, hidden_size, pred_len):
    super().__init__()
    self.rnn_layer = torch.nn.RNN(n_features, hidden_size, batch_first = True, bidirectional = True) # 양방향 적용
    self.fc_layer = torch.nn.Linear(hidden_size * 2, hidden_size) # 양방향이므로 hidden_size * 2
    self.relu = torch.nn.ReLU()
    self.output_layer = torch.nn.Linear(hidden_size, pred_len) # 예측 일수: pred_len

  def forward(self, x):
    # output > batch, seq, feature
    # hn > nlayer, batch, feature
    output, hn = self.rnn_layer(x)

    # nlayer, batch, feature > batch, nlayer, feature > batch, nlayer*feature(flatten)
    x = hn.permute(1,0,2).flatten(1)

    x = self.fc_layer(x)
    x = self.relu(x)
    return self.output_layer(x)

In [ ]:
model = Net(x_arr.shape[2], 32, 5)
model(batch['x'])

tensor([[-0.1295,  0.1845, -0.1554,  0.1506,  0.1446]],
       grad_fn=<AddmmBackward0>)

# 테스트데이터셋 구성
- 2024-01-01~2024-03-31 기간동안 매일 주식가격을 예측했다고 가정

In [ ]:
test_df = web.DataReader('005930', 'naver', start='2024-01-01', end='2024-03-31') # 종목번호: 005930
test_df = test_df.astype(int)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 61 entries, 2024-01-02 to 2024-03-29
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Open    61 non-null     int64
 1   High    61 non-null     int64
 2   Low     61 non-null     int64
 3   Close   61 non-null     int64
 4   Volume  61 non-null     int64
dtypes: int64(5)
memory usage: 2.9 KB


In [ ]:
test_df.head()

,Open,High,Low,Close,Volume
Date,,,,,
2024-01-02,78200,79800,78200,79600,17142847
2024-01-03,78500,78800,77000,77000,21753644
2024-01-04,76100,77300,76100,76600,15324439
2024-01-05,76700,77100,76400,76600,11304316
2024-01-08,77000,77500,76400,76500,11088724


In [ ]:
df.iloc[-10:]

,Open,High,Low,Close,Volume
Date,,,,,
2023-12-14,74100,74300,72500,73100,27567593
2023-12-15,73800,74000,73200,73300,15419815
2023-12-18,73300,73400,72800,72900,9690551
2023-12-19,73000,73400,72800,73400,8907632
2023-12-20,74200,74900,73800,74800,16870156
2023-12-21,74600,75000,74300,75000,13478766
2023-12-22,75800,76300,75400,75900,14515608
2023-12-26,76100,76700,75700,76600,13164909
2023-12-27,76700,78000,76500,78000,20651042


In [ ]:
test_df = pd.concat([df.iloc[-10:], test_df], axis = 0) # seq_len = 10, pred_len= 5
test_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 71 entries, 2023-12-14 to 2024-03-29
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Open    71 non-null     int64
 1   High    71 non-null     int64
 2   Low     71 non-null     int64
 3   Close   71 non-null     int64
 4   Volume  71 non-null     int64
dtypes: int64(5)
memory usage: 3.3 KB


In [ ]:
test_data = test_df.to_numpy()
x_test, y_test = transform_data(test_data, mins, sizes)
x_test.shape, y_test.shape

((57, 10, 5), (57, 5))

In [ ]:
test_dt = FinanceDataset(x_test)
test_dl = torch.utils.data.DataLoader(test_dt, batch_size = batch_size, shuffle = False)

In [ ]:
pred_list = []

for i in range(n_splits):
  model = Net(n_features, hidden_size, pred_len)
  state_dict = torch.load(f'model_{i}.pt', weights_only= True) # 가중치만 불러오기
  model.load_state_dict(state_dict)

  _, pred = test_loop(test_dl, model, None, device)
  pred_list.append(pred)

pred = np.mean(pred_list, axis = 0)
pred.shape

(57, 5)

In [ ]:
pred = pred * sizes[3] + mins[3] # 종가이므로 인덱스: 3
true = y_test * sizes[3] + mins[3] # 종가이므로 인덱스: 3
mean_absolute_error(true, pred)

1691232450.45614